In [1]:
import openai


In [2]:
client = openai.Client()


In [3]:
assistant = client.beta.assistants.create(
    name="Tutor de Tecnologia",
    instructions="Você é um tutor sobre assuntos relacionados a tecnologia",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)


In [5]:
pergunta = """
 Em um sistema de cache com mapeamento direto, dado uma memória principal de uma cache
 de 256 KB e blocos de 64 bytes:
 - Quantas linhas existem na cache?
 - Onde o bloco de endereço 0x1A23C será mapeado na cache?
"""


In [6]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)


In [7]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)


In [8]:
run.status


'queued'

In [9]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [10]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")


SyncCursorPage[Message](data=[Message(id='msg_cPGFand8FFRcK9DgEJ31Kr6X', assistant_id='asst_u8KcwgMDaOhR8WWoH8mTMbwy', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.\n\nPortanto, para um sistema de cache com mapeamento direto com uma cache de 256 KB e blocos de 64 bytes, temos:\n- 4096 linhas na cache\n- O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.'), type='text')], created_at=1745279376, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_9lmT4xdyIvqfXWHGAsf4eTTJ', status=None, thread_id='thread_81JCfaUM3pbTwTP9MbjOkLuV'), Message(id='msg_Cb5byKYoR9WpVpouHxLj6knm', assistant_id='asst_u8KcwgMDaOhR8WWoH8mTMbwy', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O número de linhas na cache é de 4096.\n\nAgora, para determinar onde o bloco de endereço 0x1A

In [11]:
print(mensagens.data[0].content[0].text.value)


O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.

Portanto, para um sistema de cache com mapeamento direto com uma cache de 256 KB e blocos de 64 bytes, temos:
- 4096 linhas na cache
- O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.


In [12]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)


In [13]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        
        print(message.content[0].text.value)



====Step message_creation
Para um sistema de cache com mapeamento direto, precisamos calcular o número de linhas na cache e determinar onde um determinado bloco de endereço será mapeado na cache.

Dado:
- Tamanho da cache = 256 KB = 256 * 1024 bytes
- Tamanho do bloco = 64 bytes

Vamos primeiro calcular o número de linhas na cache. Isso pode ser feito dividindo o tamanho da cache pelo tamanho do bloco. Em seguida, calcularemos onde o bloco de endereço 0x1A23C será mapeado na cache.

Vamos realizar esses cálculos:

====Step tool_calls
# Tamanho da cache em bytes
tamanho_cache_bytes = 256 * 1024 

# Tamanho do bloco em bytes
tamanho_bloco_bytes = 64

# Calculando o número de linhas na cache
numero_linhas_cache = tamanho_cache_bytes / tamanho_bloco_bytes

numero_linhas_cache

====Step message_creation
O número de linhas na cache é de 4096.

Agora, para determinar onde o bloco de endereço 0x1A23C será mapeado na cache, precisamos considerar a política de mapeamento direto. 

Na política d